# Imports

In [7]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import mplfinance as mpf
import os
import matplotlib.ticker as  ticker
import base64
import openai
from openai import OpenAI
import pandas as pd
import os
import re
import json
import time
import pandas as pd

# Funções

In [8]:
# Chave APIapinumber=''
client = OpenAI(api_key=apinumber)

def encode_image_to_base64(image_path: str) -> str:
    """
    Codifica um arquivo de imagem em uma string Base64.

    Args:
        image_path: O caminho para o arquivo de imagem.

    Returns:
        A string Base64 codificada.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# --- Função principal para descrever a imagem ---
def analisar_grafico(image_path: str, prompt_comando: str) -> str | None:
    """
    Obtém uma descrição ou análise de uma imagem local usando a API de Chat Multimodal da OpenAI.

    Args:
        image_path: O caminho para o arquivo de imagem.
        prompt_comando: O comando ou pergunta a ser feita sobre a imagem.

    Returns:
        A descrição ou análise da imagem como uma string, ou None em caso de erro.
    """
    # Inicializa o cliente OpenAI. A chave da API é lida automaticamente da variável de ambiente OPENAI_API_KEY.
    # Se você precisar de um cliente específico para esta função, pode passá-lo como argumento
    # ou inicializá-lo aqui, mas o ideal é que seja inicializado uma vez fora da função se for reusado.

    try:
        print(f"Codificando imagem: {image_path}...")
        base64_image = encode_image_to_base64(image_path)
        print("Imagem codificada com sucesso.")

        messages_list = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "auto"  # 'auto' é um bom padrão, pode ser 'low' ou 'high' se necessário
                        },
                    },
                    {"type": "text", "text": prompt_comando},
                ],
            }
        ]

        print("Enviando requisição para a API da OpenAI...")
        response = client.chat.completions.create(
            model="o4-mini",
            messages=messages_list,
            temperature=1
        )
        print("-" * 20)
        print("Resposta recebida da API.")

        description = response.choices[0].message.content
        return description

    except FileNotFoundError as e:
        print(f"Erro: {e}. Verifique o caminho do arquivo.")
        return None
    except IOError as e: # Captura erros específicos de leitura de arquivo da função encode_image_to_base64
        print(f"Erro de I/O: {e}")
        return None
    except APIError as e: # Captura erros da API OpenAI, como autenticação inválida ou problemas de requisição
        print(f"Erro da API OpenAI: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

# Aplicando execução de trades

- Prompt superfcial

In [10]:
prompt = '''
# Contexto
Você é um analista de investimentos CNPI-T, especializado em análise técnica e gráfica, com vasta experiência de mercado. 
Seu parecer deve ser técnico, objetivo, isento de vieses e focado em probabilidades baseadas em evidências gráficas.

# Objetivo
Analisar o gráfico na imagem e gerar uma recomendação de trade (comprar, vender ou aguardar), preenchendo todos os campos do JSON de saída.

# Formato de saída (JSON)
{
  "recomendacao": "comprar" | "vender" | "aguardar",
  "preco_entrada": "0.00",
  "alvo_lucro": "0.00",
  "stop_loss": "0.00",
  "justificativa_tecnica": "A recomendação se baseia em..."
}'''

- Prompt detalhado

In [3]:
prompt = '''
# Contexto
Você é um analista de investimentos CNPI-T, especializado em análise técnica e gráfica, com vasta experiência de mercado. Seu parecer deve ser técnico, objetivo, isento de vieses e focado em probabilidades baseadas em evidências gráficas.

# Objetivo
Analisar o gráfico de velas (candlestick) e gerar uma recomendação de trade (comprar, vender ou aguardar), preenchendo todos os campos do JSON de saída.

# CONTEXTO DA ANÁLISE
- **Período Gráfico:** Diário (D1)
- **Janela Temporal:** Últimos 6 meses
- **Indicadores Técnicos Fornecidos:**
  - Média Móvel Simples (MMS) de 20 períodos.
  - Bandas de Bollinger (20 períodos, 2 desvios padrão).
  - Índice de Força Relativa (IFR) de 14 períodos.
  - MACD (12, 26, 9). *Nota: Utilizando a configuração padrão de mercado.*

# METODOLOGIA DE ANÁLISE (Raciocínio Passo a Passo Obrigatório)
Execute a análise seguindo rigorosamente esta sequência lógica antes de formular sua resposta final.

**1. Análise da Estrutura de Preço (Price Action):**
   - **Tendência:** Identifique a tendência principal (macro, nos 6 meses), a secundária (médio prazo de 3 meses) e a de curto prazo (micro, últimas semanas). Use topos e fundos ascendentes/descendentes para definir (Observar no gráfico topos e fundos ascendentes para tendência de alta, ou descendentes para tendência de baixa).
   - **Níveis Chave:** Mapeie as zonas de Suporte e Resistência mais relevantes no gráfico diário (Verificar no gráfico regiões com alta concentração de topos ou fundos para marcar as zonas de suporte e resistência).

**2. Análise de Indicadores Técnicos:**
   - **Tendência e Momento:**
     - **MMS 20:** O preço está trabalhando acima ou abaixo da média? Qual a inclinação da média? (Exemplo: O preço está acima da média, que apresenta forte inclinação ascendente, indicando uma forte tendência de alta.)
     - **MACD:** Analise o histograma. Um histograma acima de zero (positivo) sinaliza momento de alta; abaixo de zero (negativo), sinaliza momento de baixa.
   - **Volatilidade:**
     - **Bandas de Bollinger:** Verifique se o preço está próximo da Banda de Bollinger superior ou inferior. A proximidade das bandas pode indicar que o preço está "esticado", oferecendo potenciais pontos de entrada em pullbacks. Por exemplo, se a intenção é comprar, a proximidade da banda inferior pode ser um bom ponto de entrada; já a proximidade da banda superior pode apresentar uma oportunidade de venda.
   - **Força Relativa:**
     - **IFR 14:** O indicador está em zona de sobrecompra (>70), sobrevenda (<30) ou em território neutro? Busque oportunidades de compra quando o IFR estiver em zona de sobrevenda (<30) e de venda quando estiver em zona de sobrecompra (>70).

**3. Síntese e Formulação da Tese de Trade:**
   - **Confluência de Sinais:** Combine as informações do Price Action e dos indicadores. Há uma confluência de sinais apontando para uma direção? (Ex: "O ativo está em tendência macro e secundária de alta, mas em microtendência de baixa. O IFR está abaixo de 30, o MACD está acima de zero e o preço está entre as Bandas de Bollinger. Diante disso, a estratégia é entrar comprado no rompimento do candle anterior, seguindo a tendência principal.").
   - **Possíveis movimentações:** Avalie possíveis projeções de movimento aplicando a análise técnica na imagem (Ex: "Se o preço romper o último topo, isso pode gerar urgência compradora e impulsionar o preço até a próxima zona de resistência, tornando-se uma boa oportunidade de compra, pois há potencial de um movimento expressivo a favor da tendência.").
    
**4. Definição de Parâmetros e Gestão de Risco:**
   - **Tipo de Ordem:** Decida se irá comprar, vender ou esperar com base na análise dos indicadores técnicos, levando em consideração a direção dos fractais do mercado, a confluência dos indicadores e a região em que o preço se encontra. (Ex: "A tendência macro, secundária e micro é de baixa. O RSI está acima de 70, o preço não está próximo da banda inferior, o MACD está abaixo de zero e a média móvel está inclinada para baixo. Portanto, irei vender o ativo!").
   - **Preço de Entrada:** Defina o nível de preço exato para a entrada. (Ex: "Se o preço romper a máxima do candle anterior, há grande potencial de subir até a região de resistência, então a entrada será no rompimento dessa máxima.").
   - **Stop Loss:** Defina o preço que invalida a tese do trade. (Ex: "O ponto que invalida a tese do trade é o rompimento do último fundo, portanto, o stop será posicionado abaixo dele.").
   - **Alvo (Gain):** Defina o preço para realização de lucro. (Ex: "O alvo será posicionado na Banda de Bollinger superior, pois há grande probabilidade de o preço atingi-la.").
   - **Relação Risco/Retorno:** Calcule a relação entre o potencial de lucro e o de perda. (Ex: "A operação apresenta uma relação risco/retorno de 0.5 para 1, comparando o ganho potencial com a perda possível.").

**5. Decisão Final:**
   - Com base na análise técnica, na estrutura de preço, no gerenciamento de risco, nos movimentos prováveis e nos pontos de negociação, pondere os fatores favoráveis e contrários ao trade. Após reanalisar e validar todas as informações, se a negociação for considerada válida, retorne o JSON preenchido com todos os parâmetros.

# EXEMPLO COMPLETO DE ANÁLISE
   - Iniciamos analisando a estrutura de preço: observamos que a tendência macro é de baixa, pois está formando topos e fundos descendentes; a tendência secundária também é de baixa e a microtendência é de alta. Agora, vamos identificar regiões de suporte e resistência, observando a concentração de topos e fundos em determinadas regiões que servirão de referência para o trade. Após essa análise inicial, vamos observar os indicadores um a um e interpretar seus sinais. Observando o gráfico, vemos que o preço está acima da média e com inclinação para cima, sinalizando uma compra. Analisando o MACD, temos que ele está abaixo de zero. Avaliando o RSI, vemos que ele está acima de 70 e, por fim, o preço está tocando a banda superior da Banda de Bollinger. Assim, temos três indicadores técnicos sinalizando venda e apenas um sinalizando compra. Cruzando essa informação com a direção predominante do mercado, damos preferência à venda do ativo. Agora, vamos definir os parâmetros da operação: o rompimento da mínima do candle anterior é um bom ponto de entrada, e iremos colocar o stop acima da última máxima. O alvo será a Banda de Bollinger inferior. Com isso, temos uma relação risco/retorno de 1.4 para 1 e uma maioria de indicadores sinalizando uma venda sólida. Portanto, vamos executar o trade!

# FORMATO DE SAÍDA (JSON OBRIGATÓRIO)
O resultado final DEVE ser um único bloco de código JSON, sem nenhum texto ou comentário adicional fora dele.

{
  "recomendacao": "comprar" | "vender" | "aguardar",
  "preco_entrada": "0.00",
  "alvo_lucro": "0.00",
  "stop_loss": "0.00",
  "justificativa_tecnica": "A recomendação se baseia na confluência dos seguintes fatores: [descrever de forma concisa os pontos da análise, como a tendência, o suporte/resistência testado, os sinais dos indicadores IFR, MACD e Bandas de Bollinger que suportam a decisão]."
}'''

In [11]:
data = pd.DataFrame(columns=['recomendacao','preco_entrada','alvo_lucro','stop_loss','justificativa_tecnica','data','count'])
data

,recomendacao,preco_entrada,alvo_lucro,stop_loss,justificativa_tecnica,data,count


In [ ]:
pastas = r'C:\Users\netinhoklz\Projetos pessoais\Artigo Mestrado\ChatGPT como Analista CNPI\graficos_TSLA_semestral_todas_os_domingos'
lista_imagens = os.listdir(r'C:\Users\netinhoklz\Projetos pessoais\Artigo Mestrado\ChatGPT como Analista CNPI\graficos_TSLA_semestral_todas_os_domingos')
for caminho_imagem in lista_imagens:
    for count in range(1):
        try:
            caminho_imagem_pasta = pastas + '\\' + caminho_imagem
            analise = analisar_grafico(caminho_imagem_pasta,prompt)
            match = re.search(r'(\{.*\})', analise, re.DOTALL)
            resultado = match.group(1)
            dicionario_resultado = json.loads(resultado)
            data_imagem = caminho_imagem.replace('.jpg','')
            # data_imagem = data_imagem.replace('_','-')
            dicionario_resultado['data'] = data_imagem
            dicionario_resultado['count'] = count
            print(dicionario_resultado)
            data.loc[len(data)] = dicionario_resultado
        except:
            continue

Codificando imagem: C:\Users\netinhoklz\Projetos pessoais\Artigo Mestrado\ChatGPT como Analista CNPI\graficos_TSLA_semestral_todas_os_domingos\2020_10_11.jpg...
Imagem codificada com sucesso.
Enviando requisição para a API da OpenAI...
--------------------
Resposta recebida da API.
{'recomendacao': 'aguardar', 'preco_entrada': '150.00', 'alvo_lucro': '165.00', 'stop_loss': '140.00', 'justificativa_tecnica': 'O preço vem consolidando próximo à média móvel de 20 períodos (atualmente em torno de 144–146), com as Bandas de Bollinger achatadas indicando baixa volatilidade. O RSI está neutro em torno de 50, sem viés claro de sobrecompra ou sobrevenda. O MACD se encontra próximo da linha zero, com o histograma achatado após recente cruzamento levemente negativo, sugerindo ausência de força direcional definida. Recomenda-se aguardar um fechamento acima de 150 para confirmação de rompimento de resistência e ativação de sinal de compra, limitando o risco com stop em 140 e projetando alvo em 165,

In [ ]:
data

In [ ]:
data.to_csv('TSLA_backteste_treidando_apenas_nas_segundas_2020_01_01_ate_2025_06_15_meses_o4_mini_backteste3_PROMPT_RUIM.csv',sep = ';')